In [127]:
import pandas as pd

In [128]:
import json
import os
import csv

In [129]:
os.getcwd()

'/repos/EHDP-data/indicators'

In [130]:
test = json.load(open("data/2120.json"))

In [131]:
df = pd.DataFrame.from_dict(test)

In [200]:
def read_csv_as_index(csv_file):
    index = {}
    with open(csv_file, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            index[row['GeoID']] = row
    return index

In [201]:
csv_file = '../geography/GeoLookup.csv'

In [273]:
topojson_file = '../geography/CD.topo.json'

In [274]:
with open(topojson_file, 'r') as file:
        topojson_data = json.load(file)

In [204]:
file_list = ['data/2120.json', 'data/2121.json', 'data/2122.json', 'data/2124.json', 'data/2132.json']

In [255]:
index = read_csv_as_index(csv_file)

In [277]:
json_file = 'data/2120.json'
output_file = 'joined_data_2120.topojson'

def join_json_to_topojson(json_file, topojson_file, csv_file, output_file):
    # Step 1: Read the CSV file and store index values
    index = read_csv_as_index(csv_file)

    # Step 2: Read the JSON file
    with open(json_file, 'r') as file:
        json_data = json.load(file)

    # Step 3: Read the TopoJSON file
    with open(topojson_file, 'r') as file:
        topojson_data = json.load(file)

    # Step 4: Iterate through objects in TopoJSON
    for object_key in topojson_data['objects']:
        object_data = topojson_data['objects'][object_key]
        if object_data['type'] == 'GeometryCollection':
            # Iterate through geometries in GeometryCollection
            for geometry in object_data['geometries']:
                # Step 5: Retrieve corresponding data from JSON using index values
                id_value = geometry['properties']['GEOCODE']
                if id_value in index:
                    json_entry = index[id_value]
                    json_entry['value'] = json_data[int(id_value)]['Value']
                    json_entry['measure_id'] = json_data[int(id_value)]['MeasureID']
                    json_entry['time'] = json_data[int(id_value)]['Time']
                    # Step 6: Add retrieved data to TopoJSON properties
                    geometry['properties'] = json_entry

    # Step 7: Write updated TopoJSON data to a new file
    with open(output_file, 'w') as file:
        json.dump(topojson_data, file)

In [278]:
join_json_to_topojson(json_file, topojson_file, csv_file, output_file)


## Checking data file for indicators

In [161]:
df['GeoID'].unique()

array([  1,   2,   3,   4,   5, 101, 102, 103, 104, 105, 106, 107, 201,
       202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 301, 302, 303,
       304, 305, 306, 307, 308, 309, 310, 401, 402, 403, 404, 405, 406,
       407, 408, 409, 410, 501, 502, 503, 504])

In [162]:
df.head()

,MeasureID,GeoType,GeoID,Time,ban_summary_flag,Value,CI,Note,DisplayValue
0,651,Borough,1,2015-2017,0,21.0000,,,21.0
1,651,Borough,1,2012-2014,0,15.4881,,,15.5
2,651,Borough,1,2009-2011,0,18.8000,,,18.8
3,651,Borough,1,2005-2007,0,31.3000,,,31.3
4,651,Borough,2,2015-2017,0,18.2000,,,18.2


## Read joined data to check

In [279]:
joined = json.load(open("joined_data_2120.topojson"))

In [280]:
properties = [geometry['properties'] for geometry in joined['objects']['collection']['geometries']]

In [281]:
joined_df = pd.DataFrame(properties)

In [282]:
joined_df.head()

,GEOCODE,GEONAME,BOROUGH
0,101,Financial District (CD1),Manhattan
1,102,Greenwich Village and Soho (CD2),Manhattan
2,103,Lower East Side and Chinatown (CD3),Manhattan
3,104,Clinton and Chelsea (CD4),Manhattan
4,105,Midtown (CD5),Manhattan


## Checking Topojson file

In [191]:
properties2 = [geometry['properties'] for geometry in topojson_data['objects']['collection']['geometries']]

In [197]:
topojson_data['objects']['collection']['geometries'][0]['properties']['GEOCODE']

101

In [192]:
topo_data = pd.DataFrame(properties2)

In [193]:
topo_data.head()

,GEOCODE,GEONAME,BOROUGH
0,101,Financial District (CD1),Manhattan
1,102,Greenwich Village and Soho (CD2),Manhattan
2,103,Lower East Side and Chinatown (CD3),Manhattan
3,104,Clinton and Chelsea (CD4),Manhattan
4,105,Midtown (CD5),Manhattan


In [164]:
geo = pd.read_csv(csv_file)

In [118]:
sub = geo[geo['GeoType'] == 'Subboro']

In [115]:
ones

,GeoType,GeoTypeDesc,GeoTypeShortDesc,GeoID,Name,BoroID,Borough,Lat,Long
2,Borough,Borough,Borough,2,Brooklyn,2.0,Brooklyn,40.6447,-73.9479
83,Subboro,Sub-borough/PUMA,Neighborhood (Sub-borough/PUMA),2,Morrisania/East Tremont,1.0,Bronx,40.8454,-73.8876
